In [1]:
from torch import cuda, bfloat16
import transformers
import os
from torch import cuda, bfloat16
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

model_id = 'meta-llama/Llama-2-7b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# Need auth token for these
hf_auth = os.environ.get('hf_token')
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    # quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded on cuda:0


In [2]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

In [4]:
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Generate a Python script that analyzes packet captures from N3 and N6 network interfaces. Use Wireshark to process these captures into PDML format. Parse the PDMLs to determine packet delay based on timestamps and classify packets by source and destination IPs."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1024)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Generate a Python script that analyzes packet captures from N3 and N6 network interfaces. Use Wireshark to process these captures into PDML format. Parse the PDMLs to determine packet delay based on timestamps and classify packets by source and destination IPs. [/INST]  Sure! Here is a Python script that uses the `wireshark` library to process packet captures from N3 and N6 network interfaces, convert them to PDML format, and then parse the PDMLs to determine packet delay based on timestamps and classify packets by source and destination IPs.
First, you will need to install the `wireshark` library using `pip install wireshark`.
Here is the script:
```
import os
import wireshark
import pydml
import numpy as np

# Set the path to the capture files
capture_path = '/path/to/capture/files'
# Loop through the capture files and process each one
for filename in os.listdir(capture_path):
    # Open the capture file and read it into a wireshark.File object
    file = wireshark.File(fil

In [3]:
# Run text generation pipeline with our next model
prompt = "Load network data from csv file and calculate the total number of packets and the total data volume (in bytes) over time."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1024)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] Load network data from csv file and calculate the total number of packets and the total data volume (in bytes) over time. [/INST]  To load network data from a CSV file and calculate the total number of packets and the total data volume (in bytes) over time, you can use Python and the `pandas` library. Here's an example of how to do this:
1. First, import the `pandas` library:
```
import pandas as pd
```
2. Next, load the network data from the CSV file using the `read_csv()` function:
```
data = pd.read_csv('network_data.csv')
```
3. In this example, the network data is stored in a CSV file called `network_data.csv`. The file should contain columns for the following:
* `time`: the time (in seconds) at which the packet was sent
* `packet_count`: the number of packets sent at that time
* `packet_size`: the size (in bytes) of each packet sent at that time
* `data_volume`: the total data volume (in bytes) transmitted at that time
4. To calculate the total number of packets and the

In [4]:
import json

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

input_file_path = "/home/hb/LLM-research/evaluation/5G_SPEC/5G_SPEC_QA.json" 

with open(input_file_path, "r") as input_file:
    questions_data = json.load(input_file)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=512)

generated_answers_qa = []

n_question = 0
for q_dict in questions_data:
    n_question += 1
    question = q_dict["question"]
    # Generate answer using the language model
    prompt = f"[INST]{question}[/INST]"
    generated_answer = pipe(prompt)[0]['generated_text']
    print(f"Output: {generated_answer}")
    generated_answers_qa.append(generated_answer)
    print(f"---------------------{n_question}-----------------------------")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Output: [INST]What are the key objectives and goals of 5G network specifications, and how do they differ from previous generations of wireless networks?[/INST]  The key objectives and goals of 5G network specifications are to provide a high-speed, low-latency, and highly reliable wireless communication system that can support a wide range of applications and services. Here are some of the main objectives and goals of 5G network specifications, and how they differ from previous generations of wireless networks:

1. Higher data rates: 5G aims to provide data rates that are significantly higher than those of previous generations, with peak speeds of up to 20 Gbps. This will enable users to enjoy high-quality video streaming, online gaming, and other data-intensive applications.
2. Low latency: 5G aims to reduce latency to less than 1 ms, which is significantly lower than the latency of previous generations. This will enable real-time communication and make it possible to transmit data wit

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Output: [INST]Explain the architectural components of a typical 5G network, including the roles of gNodeBs, Core Network, and Edge Computing.[/INST]  A typical 5G network consists of several architectural components that work together to provide high-speed, low-latency, and highly reliable wireless communication services. Here are the main components of a 5G network and their roles:
1. gNodeBs (Base Stations):
gNodeBs, also known as base stations, are the primary components of a 5G network that provide connectivity to mobile devices. They are responsible for transmitting and receiving data to and from devices on the cellular network. gNodeBs are typically located on tall structures such as buildings, towers, or rooftops to provide wide coverage and minimize interference.
2. Core Network:
The core network is the central component of a 5G network that connects gNodeBs and provides network services such as packet switching, mobility management, and session management. The core network is 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Output: [INST]What is the significance of mmWave frequencies in 5G, and how do they impact network coverage and capacity?[/INST]  Millimeter wave (mmWave) frequencies are a key component of 5G wireless networks, offering several advantages over traditional microwave frequencies used in previous generations of wireless technology.
Significance of mmWave frequencies in 5G:
1. Higher frequency bands: mmWave frequencies operate at higher frequency bands (24-90 GHz) compared to microwave frequencies (less than 6 GHz). This allows for more spectrum to be utilized, resulting in increased network capacity and lower latency.
2. Higher antenna density: The higher frequency bands used by mmWave frequencies require a higher density of antennas to provide adequate coverage. This leads to a more dense network infrastructure, which can improve coverage and capacity in urban areas.
3. Better propagation: mmWave frequencies have a shorter wavelength, which allows them to penetrate buildings and other o

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Output: [INST]Describe the concept of network slicing in 5G, and provide examples of how it benefits different industries and applications.[/INST]  Network slicing is a key concept in 5G wireless networks that allows multiple independent network instances to be created on top of a shared physical infrastructure. This concept enables the creation of customized networks tailored to specific services, applications, or industries, while also improving resource utilization and reducing operational costs. In this answer, we will explore the concept of network slicing in 5G, its benefits, and examples of how it can be used in different industries and applications.
What is Network Slicing in 5G?
Network slicing in 5G is a network architecture concept that allows multiple logical networks to be created on top of a shared physical infrastructure. Each logical network, or slice, can have its own set of resources, such as bandwidth, latency, and security parameters, which are tailored to the speci

`Network config QA`: Pass